In [25]:
#part6
#transfare Learning
#OS help us work with folders , located files and 
import os
import tensorflow as tf
import matplotlib.pyplot as plot
import numpy as np
#to extract data from the zip file
import zipfile
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#what does this do?
#any plt plot command will now cause a figure window to open,
#and further commands can be run to update the plot.
#Some changes will not draw automatically, to force an update, use plt.draw()

%matplotlib inline
tf.__version__
#dataset extraction and preparation

dataset_path = './cats_and_dog_dataset.zip'
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

dataset_path_new = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")
#this model uses a fixed shape 
Image_shape = (128,128,3)
Base_model = tf.keras.applications.MobileNetV2(input_shape=Image_shape, include_top=False, weights="imagenet")
Base_model.summary()

#Freezing the base model

Base_model.trainable = False
Base_model.output

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(Base_model.output)

#prediction layer / output layer

Output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

#our model
model = tf.keras.models.Model(inputs=Base_model.input, outputs= Output_layer)
model.summary()

#compilation
model.compile(optimizer= tf.keras.optimizers.RMSprop(lr = 0.0001),loss = "binary_crossentropy"
              ,metrics=["accuracy"])
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)


#training the our model
train_generator = data_gen_train.flow_from_directory(train_dir,
                                                     target_size=(128,128), batch_size=128,
                                                     class_mode="binary")
valid_generator = data_gen_valid.flow_from_directory(validation_dir,
                                                     target_size=(128,128), 
                                                     batch_size=128, class_mode="binary")
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

#model evaluation
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Base_model.trainable = True
print("Number of layersin the base model: {}".format(len(Base_model.layers)))
fine_tune_at = 100
for layer in Base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)
##Final evaluation
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_13[0][0]                   
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________